# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [ ]:
!apt-get update && apt-get install -y python3-opencv

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Err:1 http://archive.ubuntu.com/ubuntu bionic InRelease                        
  Couldn't create temporary file /tmp/apt.conf.gELVW0 for passing config to apt-key
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Err:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
  Couldn't create temporary file /tmp/apt.conf.Oe6SMX for passing config to apt-key
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Err:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
  Couldn't create temporary file /tmp/apt.conf.Fuijts for passing config to apt-key
Err:3 http://security.ubuntu.com/ubuntu bionic-security InRelease              
  Couldn't create temporary file /tmp/apt.conf.HDXPvG for passing config to apt-key
Ign:5 https://developer.

In [ ]:
!pip install sklearn

In [ ]:
!pip3 install opencv-python

In [ ]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

## Set Arguments & hyperparameters

In [ ]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [ ]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 100
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

#### Train & Validation Set loader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']


## Model

In [ ]:
import torch.nn.functional as F

class custom_CNN(nn.Module):
    def __init__(self, num_classes):
        super(custom_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        
        self.model = models.resnet18(pretrained = False)
        self.fc1 = nn.Linear(in_features=25*29*29, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
        x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        output = self.softmax(x)
        
        return output

In [ ]:
import torchvision.models as models

class resnet(nn.Module):
    def __init__(self, num_classes):
        super(resnet, self).__init__()
        self.model = models.resnet18(pretrained = False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, input_img):
        x = self.model(input_img)
        x = self.softmax(x)
        return x

In [ ]:
import torchvision.models as models

class VGG(nn.Module):
    def __init__(self, num_classes):
        super(VGG, self).__init__()
        self.model = models.vgg16(pretrained = False)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, input_img):
        x = self.model(input_img)
        x = self.softmax(x)
        return x

## Utils
### EarlyStopper

In [ ]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [ ]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [ ]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


#### Load model and other utils

In [ ]:
# Load Model
model = custom_CNN(NUM_CLS).to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [ ]:
model

custom_CNN(
  (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 25, kernel_size=(5, 5), stride=(1, 1))
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

### epoch 단위 학습 진행

In [ ]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')

        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')


  0% 0/100 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7276648879051208, Acc: 0.5283993115318416, F1-Macro: 0.3896684455894982


  1% 1/100 [00:43<1:12:26, 43.90s/it]

Epoch 0, Val loss: 1.0114697515964508, Acc: 0.5692307692307692, F1-Macro: 0.47940503432494286
Epoch 1, Train loss: 0.7143304314878252, Acc: 0.5886402753872634, F1-Macro: 0.5548834629969772
Epoch 1, Val loss: 0.9856102168560028, Acc: 0.7538461538461538, F1-Macro: 0.7537878787878789
Validation loss decreased 1.0114697515964508 -> 0.9856102168560028


  2% 2/100 [01:28<1:11:53, 44.02s/it]

...

Epoch 24, Train loss: 0.04364178008917305, Acc: 1.0, F1-Macro: 1.0
Epoch 24, Val loss: 0.448990776843857, Acc: 0.7846153846153846, F1-Macro: 0.7833333333333333
Early stopping counter 9/10


 25% 25/100 [18:29<55:42, 44.57s/it]

Epoch 25, Train loss: 0.03830892711670862, Acc: 0.9982788296041308, F1-Macro: 0.998270632603189


 25% 25/100 [19:13<57:40, 46.14s/it]

Epoch 25, Val loss: 1.278981775045395, Acc: 0.7846153846153846, F1-Macro: 0.782088122605364
Early stopping counter 10/10
Early stopped


## Inference
### 모델 로드

In [ ]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [ ]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [ ]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [ ]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:01,  1.89s/it]

tensor([[2.8496e-01, 7.1504e-01],
        [9.5030e-01, 4.9700e-02],
        [9.5714e-01, 4.2856e-02],
        [2.5550e-02, 9.7445e-01],
        [8.9376e-01, 1.0624e-01],
        [1.7784e-03, 9.9822e-01],
        [7.1282e-01, 2.8718e-01],
        [6.6767e-01, 3.3233e-01],
        [9.9999e-01, 1.3064e-05],
        [2.1233e-07, 1.0000e+00],
        [4.5273e-01, 5.4727e-01],
        [9.6823e-01, 3.1767e-02],
        [6.2197e-01, 3.7803e-01],
        [9.9997e-01, 3.3238e-05],
        [9.9160e-01, 8.4013e-03],
        [9.9185e-01, 8.1460e-03],
        [3.0116e-01, 6.9884e-01],
        [9.9976e-01, 2.3866e-04],
        [3.8406e-01, 6.1594e-01],
        [9.0221e-01, 9.7786e-02],
        [9.9992e-01, 7.8333e-05],
        [9.9951e-01, 4.8790e-04],
        [9.4968e-01, 5.0321e-02],
        [1.5237e-03, 9.9848e-01],
        [9.9999e-01, 6.5779e-06],
        [9.8830e-01, 1.1700e-02],
        [9.9392e-01, 6.0802e-03],
        [9.2121e-01, 7.8791e-02],
        [1.3136e-04, 9.9987e-01],
        [7.812

2it [00:03,  1.96s/it]

tensor([[8.2045e-01, 1.7955e-01],
        [8.7954e-01, 1.2046e-01],
        [9.9963e-01, 3.6729e-04],
        [6.8823e-01, 3.1177e-01],
        [9.9689e-01, 3.1139e-03],
        [1.3373e-03, 9.9866e-01],
        [4.8014e-01, 5.1986e-01],
        [4.6657e-03, 9.9533e-01],
        [1.8192e-03, 9.9818e-01],
        [1.6287e-01, 8.3713e-01],
        [8.5652e-01, 1.4348e-01],
        [1.7854e-01, 8.2146e-01],
        [7.7612e-01, 2.2388e-01],
        [6.8202e-03, 9.9318e-01],
        [6.0109e-03, 9.9399e-01],
        [1.0000e+00, 1.0337e-06],
        [6.9059e-01, 3.0941e-01],
        [7.5101e-01, 2.4899e-01],
        [1.0097e-01, 8.9903e-01],
        [4.1496e-05, 9.9996e-01],
        [9.5062e-01, 4.9381e-02],
        [9.8404e-01, 1.5958e-02],
        [8.3088e-03, 9.9169e-01],
        [2.8566e-01, 7.1434e-01],
        [1.9654e-01, 8.0346e-01],
        [3.0874e-04, 9.9969e-01],
        [9.9995e-01, 4.5867e-05],
        [1.4687e-01, 8.5313e-01],
        [4.3711e-03, 9.9563e-01],
        [9.984

3it [00:06,  2.03s/it]

tensor([[2.1078e-04, 9.9979e-01],
        [3.3617e-05, 9.9997e-01],
        [7.7975e-01, 2.2025e-01],
        [8.5333e-01, 1.4667e-01],
        [2.0062e-04, 9.9980e-01],
        [9.5608e-01, 4.3922e-02],
        [9.9552e-01, 4.4804e-03],
        [9.8055e-01, 1.9447e-02],
        [9.9998e-01, 2.1166e-05],
        [9.9452e-01, 5.4777e-03],
        [2.8702e-03, 9.9713e-01],
        [1.0000e+00, 4.4893e-06],
        [9.9937e-01, 6.3109e-04],
        [9.2153e-01, 7.8468e-02],
        [6.4158e-02, 9.3584e-01],
        [4.2058e-02, 9.5794e-01],
        [5.5227e-03, 9.9448e-01],
        [9.9946e-01, 5.3716e-04],
        [9.3613e-01, 6.3870e-02],
        [3.2966e-02, 9.6703e-01],
        [2.3290e-01, 7.6710e-01],
        [6.0288e-01, 3.9712e-01],
        [4.2304e-01, 5.7696e-01],
        [8.1663e-01, 1.8337e-01],
        [7.0508e-01, 2.9492e-01],
        [6.5726e-01, 3.4274e-01],
        [8.5060e-01, 1.4940e-01],
        [9.8476e-01, 1.5243e-02],
        [1.3310e-01, 8.6690e-01],
        [1.397

4it [00:06,  1.60s/it]

tensor([[0.0052, 0.9948],
        [0.0605, 0.9395],
        [0.0146, 0.9854],
        [0.0705, 0.9295]], device='cuda:0')


### 결과 저장

In [ ]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction_cnn+resnet18.csv', index=False)